# 03c - BQML + Vertex AI > Pipelines - automated pipelines for updating models

As time goes on change occurs:
- inputs to our models may shift in distribution compared to when the model was trained - called training-serving skew
- inputs to out models may shift over time - called prediction drift
- new inputs/features may become available
- a better model may be created

In the `03b` notebook we deployed the model built with BQML in the `03a` notebook to a Vertex AI Endpoint for online prediction.  In this notebook we will build a challenger model with the same training data, also using BQML but with a different model type - a deep neural network similar what we build in the `05` series of ntoebooks.  We will construct a Vertex AI Pipeline to orchestrate the process of building the new model, comparing to the deployed mode, and conditionally replacing the deployed model with the new one.  

This process could be triggered based on time elapsed, amount of new data, detected training-serving skew or even prediction drift by using Vertex AI Monitoring.  

### Video Walkthrough of this notebook:
Includes conversational walkthrough and more explanatory information than the notebook:

<p><center><a href="https://youtu.be/kzDd94KucBQ" target="_blank" rel="noopener noreferrer"><img src="architectures/thumbnails/playbutton/03c.png" width="50%"></a></center></p>

### Prerequisites:
-  03a - BigQuery Machine Learning (BQML) - Machine Learning with SQL
-  03b - Vertex AI + BQML - Online Predictions with BQML Models

### Overview:
- Build Custom Pipeline Components
    - Use BigQuery ML to Get Predictions and Scikit-Learn to calculate model metrics
    - Use BigQuery ML to train a new model - A Deep Neural Network
    - Compare model metrics for baseline and challenger model
    - Export BigQuery ML model to Google Cloud Storage
    - Replace a model deployed to an endpoint (`03b`) with the challenger model, undeploy previous model
- Define the Pipeline Flow
- Compile the Pipeline
- Run the Pipeline in Vertex AI
- Get Predictions from the upated Endpoint

### Resources:
-  [Export formats for BigQuery ML models](https://cloud.google.com/bigquery-ml/docs/exporting-models)
-  [Python Client for Vertex AI](https://googleapis.dev/python/aiplatform/latest/aiplatform.html)
-  Codelab: [Vertex AI Pipelines Introduction](https://codelabs.developers.google.com/vertex-mlmd-pipelines#0)
-  [Vertex AI Prebuilt KFP Components](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-0.2.0/)


---
## Vertex AI - Conceptual Flow

<img src="architectures/slides/03c_arch.png">

---
## Vertex AI - Workflow

<img src="architectures/slides/03c_console.png">

---
## Setup

inputs:

In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [2]:
REGION = 'us-central1'
DATANAME = 'fraud'
NOTEBOOK = '03c'

# Resources
DEPLOY_IMAGE='us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-3:latest'
DEPLOY_COMPUTE = 'n1-standard-4'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters

packages:

In [3]:
from google.cloud import aiplatform
from datetime import datetime
from typing import NamedTuple
import kfp # used for dsl.pipeline
import kfp.v2.dsl as dsl # used for dsl.component, dsl.Output, dsl.Input, dsl.Artifact, dsl.Model, ...
from google_cloud_pipeline_components import aiplatform as gcc_aip

from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

clients:

In [4]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bq = bigquery.Client()

parameters:

In [5]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{DATANAME}/models/{NOTEBOOK}"
DIR = f"temp/{NOTEBOOK}"

In [6]:
# Give service account roles/storage.objectAdmin permissions
# Console > IMA > Select Account <projectnumber>-compute@developer.gserviceaccount.com > edit - give role
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'1026793852137-compute@developer.gserviceaccount.com'

environment:

In [7]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Custom Components (KFP)

Vertex AI Pipelines are made up of components that run independently with inputs and outputs that connect to form a graph - the pipeline.  For this notebook workflow the following custom components are used to orchestrate the training of a challenger model, evaluating the challenger and an existing model, comparing them based on model metrics, if the challenger is better then replace the model already deployed on an existing endpoint.  These custom components are constructed as python functions!

### Model Metrics
- Get Predictions for Test data from BigQuery Model
- Calculate [average precision for the precision-recall curve](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html#sklearn.metrics.average_precision_score)

In [36]:
@dsl.component(
    base_image = 'python:3.9',
    packages_to_install = ['pandas','db-dtypes','pyarrow','sklearn','google-cloud-bigquery']
)
def bqml_eval(
    project: str,
    var_target: str,
    model: str,
    dataname: str,
    metrics: dsl.Output[dsl.Metrics],
    metricsc: dsl.Output[dsl.ClassificationMetrics]
) -> NamedTuple("model_eval", [("metric", float)]):

    from collections import namedtuple
    from sklearn.metrics import average_precision_score, confusion_matrix
    from google.cloud import bigquery
    bq = bigquery.Client(project = project)

    query = f"""
    SELECT {var_target}, predicted_{var_target}, prob, splits 
    FROM ML.PREDICT (MODEL `{project}.{dataname}.{model}`,(
        SELECT *
        FROM `{project}.{dataname}.{dataname}_prepped`
        WHERE splits = 'TEST')
      ), UNNEST(predicted_{var_target}_probs)
    WHERE label=1
    """
    pred = bq.query(query = query).to_dataframe()

    auPRC = average_precision_score(pred[var_target].astype(int), pred['prob'], average='micro')    
    metrics.log_metric('auPRC', auPRC)
    metricsc.log_confusion_matrix(['Not Fraud', 'Fraud'], confusion_matrix(pred[var_target].astype(int), pred[f'predicted_{var_target}'].astype(int)).tolist())
    
    model_eval = namedtuple("model_eval", ["metric"])
    return model_eval(metric = float(auPRC))

### BigQuery - Train DNN

In [37]:
@dsl.component(
    base_image = 'python:3.9',
    packages_to_install = ['google-cloud-bigquery']
)
def bqml_dnn(
    project: str,
    var_target: str,
    var_omit: str,
    model: str,
    dataname: str,
    bqml_model: dsl.Output[dsl.Artifact]
) -> NamedTuple("bqml_training", [("query", str)]):
    
    from collections import namedtuple
    from google.cloud import bigquery
    bq = bigquery.Client(project = project)
    
    query = f"""
    CREATE OR REPLACE MODEL `{project}.{dataname}.{model}`
    OPTIONS
        (model_type = 'DNN_CLASSIFIER',
            auto_class_weights = FALSE,
            input_label_cols = ['{var_target}'],
            data_split_col = 'custom_splits',
            data_split_method = 'CUSTOM',
            EARLY_STOP = FALSE,
            OPTIMIZER = 'SGD',
            HIDDEN_UNITS = [2],
            LEARN_RATE = 0.001,
            ACTIVATION_FN = 'SIGMOID',
            MAX_ITERATIONS = 10,
            HPARAM_TUNING_ALGORITHM = 'VIZIER_DEFAULT',
            HPARAM_TUNING_OBJECTIVES = ['ROC_AUC'],
            DROPOUT = HPARAM_RANGE(0, 0.8),
            BATCH_SIZE = HPARAM_RANGE(8, 500),
            MAX_PARALLEL_TRIALS = 2,
            NUM_TRIALS = 20
        ) AS
    SELECT * EXCEPT({','.join(var_omit.split())}, splits),
        CASE
            WHEN splits = 'VALIDATE' THEN 'EVAL'
            ELSE splits
        END AS custom_splits
    FROM `{project}.{dataname}.{dataname}_prepped`
    WHERE splits != 'TEST'
    """
    job = bq.query(query = query)
    job.result()
    bqml_model.uri = f"bq://{project}.{dataname}.{model}"
    
    result = namedtuple("bqml_training", ["query"])
                
    return result(query = str(query))

### Compare Models

In [38]:
@dsl.component
def model_compare(
    base_metric: float,
    challenger_metric: float,
) -> bool: 
    
    if base_metric < challenger_metric:
        replace = True
    else:
        replace = False
    
    return replace

### Export BQML Model

In [39]:
@dsl.component(
    base_image = 'python:3.9',
    packages_to_install = ['google-cloud-bigquery']
)
def bqml_export(
    project: str,
    export_location: str,
    bqml_model: dsl.Input[dsl.Model],
    tf_model: dsl.Output[dsl.Artifact],  
):
    
    from google.cloud import bigquery
    bq = bigquery.Client(project = project)
    
    bqml_model_name = bqml_model.uri.split("/")[-1]
    export = bq.query(query = f"EXPORT MODEL `{bqml_model_name}` OPTIONS(URI = '{export_location}')")
    export.result()
    
    tf_model.uri = export_location

### Replace Model On Endpoint

In [40]:
@dsl.component(
    packages_to_install = ['google-cloud-aiplatform'],
    base_image = 'python:3.9'
)
def endpoint_update(
    project: str,
    region: str,
    endpoint_prefix: str,
    newmodel: dsl.Input[dsl.Model],
    display_name: str,
    deploy_machine: str,
    deploy_container: str,
    label: str
):
    
    from google.cloud import aiplatform
    aiplatform.init(project = project, location = region)

    # upload new model (03c)
    model = aiplatform.Model.upload(
        display_name = display_name,
        serving_container_image_uri = deploy_container,
        artifact_uri = newmodel.uri,
        labels = {'notebook':f'{label}'}
    )
    
    # find endpoint from notebook 03b
    for e in aiplatform.Endpoint.list():
        if e.display_name.startswith(endpoint_prefix): endpoint = e
    print(endpoint.display_name)

    # list model(s) on 03b endpoint
    models = endpoint.list_models()
    if len(models) == 1:
        oldmodel = models[0]
    print(oldmodel)
    
    # deploy 03c model to endpoint with traffic_split = 100
    endpoint.deploy(
        model = model,
        deployed_model_display_name = display_name,
        traffic_percentage = 100,
        machine_type = deploy_machine,
        min_replica_count = 1,
        max_replica_count = 1
    )
    
    # undeploy 03b model
    endpoint.undeploy(
        deployed_model_id = oldmodel.id
    )

---
## Pipeline (KFP) Definition

In [41]:
@dsl.pipeline(
    name = f'kfp-{NOTEBOOK}-{DATANAME}-{TIMESTAMP}', 
    pipeline_root = URI+'/'+str(TIMESTAMP)+'/kfp/'
)
def pipeline(
    endpoint_prefix: str,
    project: str = PROJECT_ID,
    region: str = REGION,
    dataname: str = DATANAME,
    display_name: str = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    deploy_machine: str = DEPLOY_COMPUTE,
    deploy_container: str = DEPLOY_IMAGE,
    bq_source: str = f'bq://{PROJECT_ID}.{DATANAME}.{DATANAME}_prepped',
    var_target: str = VAR_TARGET,
    var_omit: str = VAR_OMIT,
    uri: str = URI,
    label: str = NOTEBOOK 
):
        
    # get AUC for current model
    base_model_eval = bqml_eval(
        project = project,
        var_target = var_target,
        model = f'{dataname}_lr',
        dataname = dataname
    )
    base_model_eval.set_caching_options(False)
    
    # train challenger model with BQML
    challenger_model = bqml_dnn(
        project = project,
        var_target = var_target,
        var_omit = var_omit,
        model = f'{dataname}_dnn',
        dataname = dataname,
    )
    challenger_model.set_caching_options(True)
    
    # get AUC for challenger model
    challenger_model_eval = bqml_eval(
        project = project,
        var_target = var_target,
        model = f'{dataname}_dnn',
        dataname = dataname
    )
    challenger_model_eval.set_caching_options(False)
    challenger_model_eval.after(challenger_model)
    
    # compare models
    compare = model_compare(
        base_metric = base_model_eval.outputs["metric"],
        challenger_metric = challenger_model_eval.outputs["metric"]
    )
    
    # conditional deployment
    with dsl.Condition(
        compare.output == 'true',
        name = "replace_model"
    ):
        # export BQML model
        export = bqml_export(
            project = project,
            export_location = uri,
            bqml_model = challenger_model.outputs["bqml_model"]
        )
        
        # replace model on endpoint (03b)
        replace = endpoint_update(
            project = project,
            region = region,
            endpoint_prefix = endpoint_prefix,
            newmodel = export.outputs["tf_model"],
            display_name = display_name,
            deploy_machine = deploy_machine,
            deploy_container = deploy_container,
            label = label
        )

---
## Compile Pipeline

In [42]:
kfp.v2.compiler.Compiler().compile(
    pipeline_func = pipeline,
    package_path = f"{DIR}/{NOTEBOOK}.json"
)

/opt/conda/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1281: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


Move compiled pipeline files to GCS Bucket

In [43]:
!gsutil cp {DIR}/{NOTEBOOK}.json {URI}/{TIMESTAMP}/kfp/

Copying file://temp/03c/03c.json [Content-Type=application/json]...
/ [1 files][ 31.1 KiB/ 31.1 KiB]                                                
Operation completed over 1 objects/31.1 KiB.                                     


---
## Create Vertex AI Pipeline Job

In [44]:
pipeline = aiplatform.PipelineJob(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    template_path = f"{URI}/{TIMESTAMP}/kfp/{NOTEBOOK}.json",
    pipeline_root = f"{URI}/{TIMESTAMP}/kfp/",
    parameter_values = {"endpoint_prefix": "03b"},
    enable_caching = None,
    labels = {'notebook':f'{NOTEBOOK}'}
)

In [45]:
response = pipeline.run(service_account = SERVICE_ACCOUNT)

Creating PipelineJob
PipelineJob created. Resource name: projects/1026793852137/locations/us-central1/pipelineJobs/kfp-03c-fraud-20220707160930-20220707162146
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/1026793852137/locations/us-central1/pipelineJobs/kfp-03c-fraud-20220707160930-20220707162146')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/kfp-03c-fraud-20220707160930-20220707162146?project=1026793852137
PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/kfp-03c-fraud-20220707160930-20220707162146 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/kfp-03c-fraud-20220707160930-20220707162146 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/1026793852137/locations/us-central1/pipelineJobs/kfp-03c-fraud-20220707160930-20220707162146 current state:
PipelineState.PIPELINE_S

In [46]:
aiplatform.get_pipeline_df(pipeline = f'kfp-{NOTEBOOK}-{DATANAME}-{TIMESTAMP}')

,pipeline_name,run_name,param.input:var_omit,param.input:bq_source,param.input:label,param.input:display_name,param.input:region,param.input:dataname,param.input:deploy_container,param.input:project,param.input:uri,param.input:endpoint_prefix,param.input:deploy_machine,param.input:var_target,metric.confusionMatrix,metric.auPRC
0,kfp-03c-fraud-20220707160930,kfp-03c-fraud-20220707160930-20220707162146,transaction_id,bq://statmike-mlops-349915.fraud.fraud_prepped,03c,03c_fraud_20220707160930,us-central1,fraud,us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu...,statmike-mlops-349915,gs://statmike-mlops-349915/fraud/models/03c,03b,n1-standard-4,Class,{'annotationSpecs': [{'displayName': 'Not Frau...,0.806335
1,kfp-03c-fraud-20220707160930,kfp-03c-fraud-20220707160930-20220707161243,transaction_id,bq://statmike-mlops-349915.fraud.fraud_prepped,03c,03c_fraud_20220707160930,us-central1,fraud,us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu...,statmike-mlops-349915,gs://statmike-mlops-349915/fraud/models/03c,03b,n1-standard-4,Class,NaN,NaN


## Review Pipeline Run

<img src="architectures/notebooks/03c_screenshots/kfp_pipeline.png">

---
## Prediction

### Prepare a record for prediction: instance and parameters lists

In [47]:
pred = bq.query(query = f"SELECT * FROM {DATANAME}.{DATANAME}_prepped WHERE splits='TEST' LIMIT 10").to_dataframe()

In [48]:
pred.head(4)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,32799,1.153477,-0.047859,1.358363,1.480620,-1.222598,-0.481690,-0.654461,0.128115,0.907095,...,-0.025964,0.701843,0.417245,-0.257691,0.060115,0.035332,0.0,0,e9d16028-4b41-4753-87ee-041d33642ae9,TEST
1,35483,1.286640,0.072917,0.212182,-0.269732,-0.283961,-0.663306,-0.016385,-0.120297,-0.135962,...,0.052674,0.076792,0.209208,0.847617,-0.086559,-0.008262,0.0,0,8b319d3a-2b2d-445b-a9a2-0da3d664ec2a,TEST
2,163935,1.961967,-0.247295,-1.751841,-0.268689,0.956431,0.707211,0.020675,0.189433,0.455055,...,0.186420,-1.621368,-0.131098,0.034276,-0.004909,-0.090859,0.0,0,788afb87-60aa-4482-8b48-c924bec634aa,TEST
3,30707,-0.964364,0.176372,2.464128,2.672539,0.145676,-0.152913,-0.591983,0.305066,-0.148034,...,-0.024200,0.365226,-0.745369,-0.060544,0.095692,0.217639,0.0,0,473d0936-1974-4ae8-ab70-230e7599bd3f,TEST


In [49]:
newob = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+[VAR_TARGET, 'splits'])]].to_dict(orient='records')[0]
newob

{'Time': 32799,
 'V1': 1.15347743766561,
 'V2': -0.0478588055804616,
 'V3': 1.35836288729212,
 'V4': 1.48062009487976,
 'V5': -1.22259808550513,
 'V6': -0.481689608379461,
 'V7': -0.6544612667240159,
 'V8': 0.128114599402494,
 'V9': 0.907094671648477,
 'V10': -0.0364175882073298,
 'V11': -0.659895142180526,
 'V12': -0.307334930019039,
 'V13': -1.3656343720183501,
 'V14': 0.035305634441467997,
 'V15': 0.7114399756440071,
 'V16': 0.25384340384905,
 'V17': -0.22283165732475999,
 'V18': 0.26559908021792394,
 'V19': -0.5970199308963089,
 'V20': -0.24568562944008399,
 'V21': 0.12551444308840598,
 'V22': 0.480049308608633,
 'V23': -0.0259637518455547,
 'V24': 0.7018428515999721,
 'V25': 0.41724451503333504,
 'V26': -0.257690694038964,
 'V27': 0.06011458360133701,
 'V28': 0.0353320583585812,
 'Amount': 0.0}

### Get Predictions: Python Client
This model does not have the default signature name expected by Vertex AI endpoint of "serving_default".  To get arround this we prepare the json input to include the correct signature name and use the [raw_predict method with the PredictionService documented here](https://googleapis.dev/python/aiplatform/latest/aiplatform_v1/prediction_service.html#google.cloud.aiplatform_v1.services.prediction_service.PredictionServiceClient.raw_predict).

In [50]:
for e in aiplatform.Endpoint.list():
    if e.display_name.startswith('03b'): endpoint = e
print(endpoint.display_name)
print(endpoint.resource_name)

03b_fraud_20220707104636
projects/1026793852137/locations/us-central1/endpoints/6922833071733473280


In [51]:
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
predictor = aiplatform.gapic.PredictionServiceClient(client_options = client_options)

In [52]:
from google.api import httpbody_pb2
instances = {"instances": [newob], "signature_name": "predict"}
http_body = httpbody_pb2.HttpBody(data = json.dumps(instances).encode("utf-8"), content_type = "application/json")

In [53]:
pred = predictor.raw_predict(endpoint = endpoint.resource_name, http_body = http_body)

In [54]:
json.loads(pred.data)

{'predictions': [{'classes': ['0'],
   'all_class_ids': [0, 1],
   'logistic': [0.999906898],
   'all_classes': ['1', '0'],
   'probabilities': [9.31419345e-05, 0.999906898],
   'logits': [9.28129292],
   'class_ids': [1]}]}

### Get Predictions: REST
This model does not have the default signature name expected by Vertex AI endpoint of "serving_default".  To get arround this we prepare the json input to include the correct signature name and use the [raw predict method documented here](https://cloud.google.com/vertex-ai/docs/reference/rest/v1/projects.locations.endpoints/rawPredict).

In [55]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"signature_name": "predict", "instances": [newob]}))

In [56]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:rawPredict

{
    "predictions": [
        {
            "probabilities": [9.31419345e-05, 0.999906898],
            "logits": [9.28129292],
            "class_ids": [1],
            "classes": ["0"],
            "all_class_ids": [0, 1],
            "logistic": [0.999906898],
            "all_classes": ["1", "0"]
        }
    ]
}

### Get Predictions: gcloud (CLI)
This model does not have the default signature name expected by Vertex AI endpoint of "serving_default".  To get arround this we prepare the json input to include the correct signature name and use the [raw predict method documented here](https://cloud.google.com/sdk/gcloud/reference/ai/endpoints/raw-predict).

In [57]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"signature_name": "predict", "instances": [newob]}))

In [58]:
!gcloud ai endpoints raw-predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --request=@{DIR}/request.json

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
{
    "predictions": [
        {
            "logistic": [0.999906898],
            "all_classes": ["1", "0"],
            "probabilities": [9.31419345e-05, 0.999906898],
            "logits": [9.28129292],
            "class_ids": [1],
            "classes": ["0"],
            "all_class_ids": [0, 1]
        }
    ]
}

---
## Remove Resources
see notebook "99 - Cleanup"